In [23]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import json

In [24]:
class SemanticSearchEngine:
    def __init__(self, model_name='ricardo-filho/bert-base-portuguese-cased-nli-assin-2', use_gpu=torch.cuda.is_available()):
        self.model = SentenceTransformer(model_name)
        if use_gpu:
            self.model = self.model.to('cuda') 
        self.dimension = self.model.get_sentence_embedding_dimension()
        self.index = faiss.IndexFlatIP(self.dimension)
        self.id_to_data = {}

    def add_entries(self, entries):
        ids, descriptions = zip(*entries)
        embeddings = self.model.encode(descriptions, convert_to_tensor=True, normalize_embeddings=True)
        if torch.cuda.is_available():
            embeddings = embeddings.cpu()
        embeddings = embeddings.numpy()
        self.index.add(embeddings)
        for entry_id, description in zip(ids, descriptions):
            self.id_to_data[entry_id] = description

    def search(self, query_text, top_k=5):
        query_embedding = self.model.encode([query_text], convert_to_tensor=True, normalize_embeddings=True)
        if torch.cuda.is_available():
            query_embedding = query_embedding.cpu()
        query_embedding = query_embedding.numpy()
        distances, indices = self.index.search(query_embedding, top_k)
        results = [(list(self.id_to_data.keys())[idx], self.id_to_data[list(self.id_to_data.keys())[idx]], distance) for idx, distance in zip(indices[0], distances[0])]
        return sorted(results, key=lambda x: x[2], reverse=True)

    def save_index(self, file_path):
        faiss.write_index(self.index, file_path + '.index')
        with open(file_path + '.json', 'w') as f:
            json.dump(self.id_to_data, f)

    def load_index(self, file_path):
        self.index = faiss.read_index(file_path + '.index')
        with open(file_path + '.json', 'r') as f:
            self.id_to_data = json.load(f)

In [12]:
with open('Cursos.json', 'r') as f:
    data = json.load(f)

courses = [(entry['id'], entry['description']) for entry in data['data']]

In [14]:
engine.add_entries(courses)

In [15]:
engine.save_index('cursos')

In [27]:
results = engine.search('curso para surdos', 10)
results

[('bfae1ed6-dfa1-41e3-9793-704587b4e8ed',
  'Introdução à legislação para pessoa com deficiência. Padrões e Normas sobre acessibilidade para pessoas com deficiência intelectual. Tecnologia Assistiva para pessoa com deficiência intelectual. Material Didático Acessível à Pessoas com Deficiência Intelectual.',
  0.5925292),
 ('bae6feff-3be9-4517-b777-da1c5b489d91',
  'Conceituação de surdez e conceituação e classificação de deficiência auditiva. Principais patologias e prevenção da deficiência auditiva na infância. Formação de conceitos e construção de conhecimentos: alunos surdez ou deficiência auditiva. Aprendizagem de alunos com surdez. Procedimentos didáticos - Metodológicos: classe comum e atendimento educacional especializado: ensino em Libras, ensino de Língua Portuguesa na modalidade escrita. Tecnologias Educacionais Assistivas para surdez e deficiência auditiva.',
  0.5920518),
 ('8a5b501d-4c05-46a6-bd20-09c088d803f1',
  'A audiodescrição é um recurso de acessibilidade comunicaci